In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Simple Impact-T Run

In [2]:
from impact import Impact

In [3]:
ipath = '../templates/lcls_injector/'
ifile = '../templates/lcls_injector/ImpactT.in'

In [4]:
I = Impact(ifile, verbose=True)

================ Impact-T Summary ================
1 bunchtotal charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Tracking 300004 particles
Processor domain: 1 x 1 = 1 CPUs
Computational domain: 0.015 m x 0.015 m x 45.0 m
Space charge grid: 64 x 64 x 128
Maximum time steps: 1000000
Random Seed: 6
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning


Configured to run in: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/temp_impactT_t6zpsj_w


In [5]:
# Change some things
I.input['header']['Np'] = 10000
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

# Other switches
#I.timeout = 1000
# Switches for MPI
I.use_mpi=True
I.input['header']['Npcol'] = 4
I.impact_bin = '$IMPACTT_MPI_BIN'

In [6]:
# Change stop location
I.ele['stop_1']['s'] = 2.0

In [ ]:
I.run()

 !-----------------------------------------------------------
 ! IMPACT-T Parallel Beam Dynamics Tracking Code: 2.0 beta version
 ! Copyright of The Regents of the University of California
 !-----------------------------------------------------------
 nblem:           12          12
 pass setting up lattice...
 check randomness:            0  0.79479214134216858     
 check randomness:            1  0.65948074101936072     
 check randomness:            3  0.79369999429767613     
 check randomness:            2  0.68444898109486740     
 avgpts, jlow, and jhigh:         2500           1        2500
 avgpts, jlow, and jhigh:         2500        7501       10000
 avgpts, jlow, and jhigh:         2500        2501        5000
 avgpts, jlow, and jhigh:         2500        5001        7500
 zmin,zmax:    4.1346249090536441E-002   4.3453330768569699E-002, elapsed: 8.234201669692993

In [ ]:
I.output['slice_info']

In [ ]:
I.particles

In [ ]:
I.output['stats'].keys()

In [ ]:
I.output['slice_info']

# Plot

In [ ]:
from bokeh.plotting import output_notebook
from bokeh.plotting import show
output_notebook(hide_banner=True)

from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Range1d
from impact.parsers import load_many_fort
from impact.parsers import UNITS

from impact.plotting import layout_plot


ds = ColumnDataSource(I.output['stats'])

In [ ]:
I.ele['write_beam_1']

In [ ]:
# Change somethone
I.ele['QE01']['b1_gradient'] = 0
layout = layout_plot(I.input['lattice'], plot_height=500)
show(layout)

In [ ]:
def plot1(y_axis, x_axis='z', source=ds):
    yunit = UNITS[y_axis]
    ylabel = y_axis+' ('+yunit+')'
    
    xunit = UNITS[x_axis]
    xlabel = x_axis+' ('+xunit+')'
    p = figure(plot_width=600, plot_height=200, x_axis_label=xlabel, y_axis_label=ylabel)
    p.line( x_axis, y_axis, source=source, color='red')
    return p
show(plot1('gamma'))

# Archiving

In [ ]:
from impact import writers
import h5py

In [ ]:
# This is the total bunch charge in Coulombs
total_charge = I.input['header']['Bcurr'] / I.input['header']['Bfreq']
total_charge

In [ ]:
I.particles

In [ ]:
h5 = h5py.File('test.h5', 'w')

In [ ]:
I.archive(h5)

In [ ]:
h5.close()

# Reading archive

In [ ]:
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
import os

from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats

In [ ]:
h5 = h5py.File('test.h5', 'r')

In [ ]:
list(h5)

In [ ]:
list(h5['particles'])

In [ ]:
bunch1 = h5['particles/final_particles']
show(
    column(
        
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'x', bins = 50, liveOnly=False),
    bunch_plotting.plot_histogram_h5(bunch1, 'x', bins=100,liveOnly=False),
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'pz', bins = 50, liveOnly=False),
   # bunch_plotting.plot_bunch_current_profile(bunch1, bins=100, liveOnly=False)
    
    
    ))